# Combine Data

Data has now been filtered for the parcel numbers in the existing dataset. The end goal is to have a dataset with one row for each parcel in which to build the model.

However, we can't forget to explore the existing data for trends or insight along the way, as well as engineer features as we see fit.

In [75]:
import pandas as pd
import numpy as np

tci = pd.read_csv('TCI_surveyed2.csv')
ppns = set(tci['Parcel Number'])
tci.columns = ['parcel','survey_date','vacant']

## Add data from property characteristics

In [ ]:
prop = pd.read_csv('clean_data/main_prop_tci.csv')
prop = prop.drop_duplicates()

In [3]:
prop.columns[[5,34,36]]

Index([u'zip', u'glsflag', u'gtxyr'], dtype='object')

In [4]:
prop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186947 entries, 0 to 186946
Data columns (total 65 columns):
parcel       186947 non-null object
taxyr        186895 non-null float64
pclass       186945 non-null object
nluc         184021 non-null float64
luc          186947 non-null object
zip          146684 non-null object
owner        186888 non-null object
propsize     182319 non-null object
front        186601 non-null float64
depth        184989 non-null float64
lotshape     152179 non-null object
totbldgs     186507 non-null float64
condition    164113 non-null object
units        163429 non-null float64
area         164031 non-null object
totusabl     181893 non-null object
yrbuilt      164099 non-null float64
style        160997 non-null object
rextwall     160978 non-null object
rooms        160949 non-null float64
bedrooms     160971 non-null float64
baths        160978 non-null float64
halfbath     160974 non-null float64
bval         185571 non-null object
lval         1

### Zip code

In [ ]:
def filter_zip(x):
    try:
        x = str(int(x))
    except ValueError:
        pass
    return x

In [71]:
prop['zip'] = prop['zip'].dropna().apply(filter_zip)
zips = set(prop['zip'].dropna())
zips.difference_update(['.','0','4','441041'])
prop_zips = prop[['parcel','zip']][prop['zip'].isin(zips)].groupby('parcel').first()

In [76]:
tci = pd.merge(tci, prop_zips, how='left', left_on='parcel', right_index=True)

### Land use code

In [84]:
set(prop['propsize'].apply(type))

{float, str}

In [85]:
def parse_comma(x):
    if type(x) == 'str':
        x = int(x.replace(',',''))
    elif type(x) == 'float':
        x = int(float)
    else:
        print("something's wrong")
    return x

In [ ]:
prop['propsize'].apply(parse_comma)

something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something's wrong
something'

In [48]:
p[['parcel','luc']].groupby('luc').count()

,parcel
luc,
6106,3
6250,53
7050,119
Apartments 1 6 units,8
Apartments 20 39 units (walk up),3
"Charitable exemptions (hospitals,home for age, etc.)",30
Churches,214
Churches residential,1
Churches retail,1
